# PoC Mail Scraping API

In [1]:
!pip install --upgrade google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 6.4 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [18]:
!pip install beautifulsoup4 pandas


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [4]:
import os
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build

# Define the scope for reading Gmail emails
SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]

# Update this with your actual file name
CLIENT_SECRET_FILE = "client_secret_780078547466-momspjf4o5ge026ksnim4oj0e1r0asgg.apps.googleusercontent.com.json"
TOKEN_FILE = "token.json"

def authenticate_gmail():
    creds = None

    # Check if the token file exists (to avoid re-authentication)
    if os.path.exists(TOKEN_FILE):
        creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)

    # If credentials are invalid or not available, do OAuth2 login
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
            creds = flow.run_local_server(port=0)

        # Save the credentials for future use
        with open(TOKEN_FILE, "w") as token:
            token.write(creds.to_json())

    return build("gmail", "v1", credentials=creds)

# Authenticate and create the Gmail API service
gmail_service = authenticate_gmail()
print("✅ Gmail API Authentication Successful!")

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=780078547466-momspjf4o5ge026ksnim4oj0e1r0asgg.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A49282%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=s6wYATQp2ePRRETL4NM7lIwLgWGsmg&access_type=offline
✅ Gmail API Authentication Successful!


In [6]:
def search_emails(service, sender_email, max_results=10):
    """Search for emails from a specific sender."""
    query = f"from:{sender_email}"
    results = service.users().messages().list(userId="me", q=query, maxResults=max_results).execute()
    
    messages = results.get("messages", [])
    return messages

# Define the sender's email
sender_email = "aukehofman97@gmail.com"  # Change to your target sender

# Fetch emails from sender
emails = search_emails(gmail_service, sender_email)

# Print email IDs found
if emails:
    print("📩 Emails Found:", [email["id"] for email in emails])
else:
    print("❌ No emails found from this sender.")

📩 Emails Found: ['194f5fe95e74a426']


In [25]:
import base64
import pandas as pd
import re
from bs4 import BeautifulSoup
from IPython.display import display

def extract_table_from_email_spacing(service, email_id):
    """Extracts a table from an email using spacing-based parsing (like PDF extraction)."""
    message = service.users().messages().get(userId="me", id=email_id, format="full").execute()
    headers = message["payload"]["headers"]

    # Get subject and sender
    subject = next((h["value"] for h in headers if h["name"] == "Subject"), "No Subject")
    sender = next((h["value"] for h in headers if h["name"] == "From"), "Unknown Sender")

    # Extract body (plain text or HTML)
    parts = message["payload"].get("parts", [])
    body = None
    is_html = False

    for part in parts:
        if part["mimeType"] == "text/plain":  # Plain text
            body = base64.urlsafe_b64decode(part["body"]["data"]).decode("utf-8", errors="ignore")
            break
        elif part["mimeType"] == "text/html":  # HTML email
            body = base64.urlsafe_b64decode(part["body"]["data"]).decode("utf-8", errors="ignore")
            is_html = True
            break

    # 🛠️ If it's an HTML table, extract using BeautifulSoup
    if is_html:
        soup = BeautifulSoup(body, "html.parser")
        table = soup.find("table")  # Find the first table

        if table:
            rows = []
            for row in table.find_all("tr"):  # Iterate over table rows
                cols = [col.get_text(strip=True) for col in row.find_all(["td", "th"])]
                rows.append(cols)

            # Convert to DataFrame
            df = pd.DataFrame(rows[1:], columns=rows[0])  # First row as header
            
            # Display in Jupyter Notebook
            display(df)
            return df
        else:
            print("❌ No table found in HTML email.")

    # 🛠️ If it's plain text, process using spacing-based parsing
    elif body:
        lines = body.split("\n")  # Split email into lines

        # Find the table start (where "Kenmerk" appears)
        table_start_idx = next((i for i, line in enumerate(lines) if "Kenmerk" in line), None)

        if table_start_idx is not None:
            table_lines = lines[table_start_idx:]  # Keep only lines from "Kenmerk" onwards
            
            # Step 1: Extract Headers (First row after "Kenmerk")
            headers = re.split(r"\s{2,}", table_lines[0].strip())  # Split using multiple spaces
            headers = [h.strip() for h in headers if h.strip()]  # Remove empty elements

            # Step 2: Extract Data Rows (Using consistent spacing)
            data = []
            for line in table_lines[1:]:  # Skip header row
                row = re.split(r"\s{2,}", line.strip())  # Split using multiple spaces
                row = [r.strip() for r in row if r.strip()]  # Remove empty values

                if len(row) == len(headers):  # Ensure row matches column count
                    data.append(row)

            # Convert to DataFrame
            df = pd.DataFrame(data, columns=headers)

            # Display in Jupyter Notebook
            display(df)
            return df
        else:
            print("❌ No recognizable table found in plain text email.")

    print("📩 Subject:", subject)
    print("📧 From:", sender)
    print("📝 No structured table found in email.")
    return None

# Fetch and process the first found email
if emails:
    email_id = emails[0]["id"]
    df = extract_table_from_email_spacing(gmail_service, email_id)
else:
    print("❌ No emails found.")

,>>>> Kenmerk
0,>>>> Actie
1,>>>> Klantnr
2,>>>> Naam
3,>>>> Land
4,>>>> Straat
...,...
362,>>>> 696
363,>>>> 1
364,>>>> 44
365,>>
